In [3]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statistics
from sklearn.model_selection import train_test_split

In [21]:
y_csv = 'ABCB4.csv'
x_csv = 'AZUL4.csv'

data1 = pd.DataFrame(pd.read_csv('datasets-b3/'+y_csv)['Fechamento'], columns=['Fechamento'])
data2 = pd.DataFrame(pd.read_csv('datasets-b3/'+x_csv)['Fechamento'], columns=['Fechamento'])

data2.columns = data1.columns = ['Close']
data2.shape, data1.shape

((92, 1), (92, 1))

# Cointegration

rain test split 

In [5]:
df1,test1,df2,test2=train_test_split(data1,data2,test_size=0.7,shuffle=False)

train=pd.DataFrame()
train['asset1']=df1['Close']
train['asset2']=df2['Close']

In [19]:
train.shape

(27, 2)

In [12]:
x=sm.add_constant(train['asset1'])
y=train['asset2']
model=sm.OLS(y,x).fit()

print('\n',sm.tsa.stattools.adfuller(model.resid))
print(model.summary())


 (-2.6572890685091433, 0.08172518035680337, 0, 26, {'1%': -3.7112123008648155, '5%': -2.981246804733728, '10%': -2.6300945562130176}, 63.53380697619901)
                            OLS Regression Results                            
Dep. Variable:                 asset2   R-squared:                       0.353
Model:                            OLS   Adj. R-squared:                  0.327
Method:                 Least Squares   F-statistic:                     13.62
Date:                Sun, 17 May 2020   Prob (F-statistic):            0.00109
Time:                        14:12:22   Log-Likelihood:                -48.391
No. Observations:                  27   AIC:                             100.8
Df Residuals:                      25   BIC:                             103.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|  

signals

In [13]:
signals=pd.DataFrame()
signals['asset1']=test1['Close']
signals['asset2']=test2['Close']

In [14]:
signals['fitted']=np.mat(sm.add_constant(signals['asset2']))*np.mat(model.params).reshape(2,1)
signals['residual']=signals['asset1']-signals['fitted']
signals['z']=(signals['residual']-np.mean(signals['residual']))/np.std(signals['residual'])

In [38]:
#use z*0 to get panda series instead of an integer result
signals['z upper limit']=signals['z']*0+np.mean(signals['z'])+np.std(signals['z'])
signals['z lower limit']=signals['z']*0+np.mean(signals['z'])-np.std(signals['z'])

In [40]:
signals['asset1']-signals['fitted']

27   -71.543712
28   -72.689537
29   -73.823806
30   -73.251672
31   -73.891539
        ...    
87   -38.807470
88   -38.463646
89   -38.133779
90   -37.692911
91   -37.284865
Length: 65, dtype: float64

In [37]:
np.mat(model.params).reshape(2,1)

matrix([[38.36266564],
        [ 0.95777977]])

In [24]:
signals.head(3)

,asset1,asset2,fitted,residual,z,z upper limit,z lower limit,signals1
27,21.24,56.82,92.783712,-71.543712,-1.845965,1.0,-1.0,0
28,21.55,58.34,94.239537,-72.689537,-1.936994,1.0,-1.0,0
29,21.68,59.66,95.503806,-73.823806,-2.027105,1.0,-1.0,0


# Signal_generation

In [25]:
signals['signals1']=0
signals.head(3)

,asset1,asset2,fitted,residual,z,z upper limit,z lower limit,signals1
27,21.24,56.82,92.783712,-71.543712,-1.845965,1.0,-1.0,0
28,21.55,58.34,94.239537,-72.689537,-1.936994,1.0,-1.0,0
29,21.68,59.66,95.503806,-73.823806,-2.027105,1.0,-1.0,0


In [29]:
#as z statistics cannot exceed both upper and lower bounds at the same time
#this line holds
signals['signals1']=np.select([signals['z']>signals['z upper limit'],
                               signals['z']<signals['z lower limit']],
                               [-1,1],default=0)
signals.head(3)

,asset1,asset2,fitted,residual,z,z upper limit,z lower limit,signals1
27,21.24,56.82,92.783712,-71.543712,-1.845965,1.0,-1.0,1
28,21.55,58.34,94.239537,-72.689537,-1.936994,1.0,-1.0,1
29,21.68,59.66,95.503806,-73.823806,-2.027105,1.0,-1.0,1


In [31]:
#signals only imply holding
#we take the first order difference to obtain the execution signal
signals['positions1']=signals['signals1'].diff()
signals['signals2']=-signals['signals1']
signals['positions2']=signals['signals2'].diff()
signals.head(3)

,asset1,asset2,fitted,residual,z,z upper limit,z lower limit,signals1,positions1,signals2,positions2
27,21.24,56.82,92.783712,-71.543712,-1.845965,1.0,-1.0,1,NaN,-1,NaN
28,21.55,58.34,94.239537,-72.689537,-1.936994,1.0,-1.0,1,0.0,-1,0.0
29,21.68,59.66,95.503806,-73.823806,-2.027105,1.0,-1.0,1,0.0,-1,0.0


In [33]:
#fix initial positions issue
if signals['signals1'].iloc[0]!=0:
    signals['positions1'].iloc[0]=signals['signals1'].iloc[0]
    signals['positions2'].iloc[0]=signals['signals2'].iloc[0]  
signals.head(3)

,asset1,asset2,fitted,residual,z,z upper limit,z lower limit,signals1,positions1,signals2,positions2
27,21.24,56.82,92.783712,-71.543712,-1.845965,1.0,-1.0,1,1.0,-1,-1.0
28,21.55,58.34,94.239537,-72.689537,-1.936994,1.0,-1.0,1,0.0,-1,0.0
29,21.68,59.66,95.503806,-73.823806,-2.027105,1.0,-1.0,1,0.0,-1,0.0
